### k3d_util.pyを同じディレクトリに格納しておくこと。

In [ ]:
import os, time, re
from pathlib import Path
import numpy as np
import cv2
from PIL import Image
# Image.MAX_IMAGE_PIXELS = 2560*2160*10    # 画像サイズが大きいときはここの数値を大きくする

In [ ]:
import k3d

In [ ]:
import k3d_util as util

In [ ]:
# 元画像データの格納場所
data_path = Path('/Volumes/NVMeSSD256GB/TutorialData_Spheroid/Image')

# numpy配列データの格納場所
npz_data_path = Path('./tutorial_spheroid.npz')

In [ ]:
# 対物レンズ倍率
util.MAGNIFICATION = 40

# 撮影時のビニング
util.BINNING = 1

# Z間隔
util.Z_SLICE_LENGTH = 2.0

# 画像の画素数を減らすための画像縮小率
util.REDUCE_RATIO = 4

# Zスタックの枚数を減らすためのZステップ間隔
util.Z_STEP = 1

In [ ]:
# CV8000でZを3桁撮るとファイル名が桁あふれしてしまうため、正規表現でZ番号を取得する
def z_number(file_name):
    m = re.search(r'Z(\d+)[^Z]*', file_name)
    return int(m.groups()[0])

# 3D画像データの読み込み関数の定義
def load_data(file_list):
    file_list.sort(key=z_number)
    img = Image.open(file_list[0])
    buf = np.zeros((len(file_list[::util.Z_STEP]), img.size[1]//util.REDUCE_RATIO, img.size[0]//util.REDUCE_RATIO), dtype=np.float32)
    for z, i in enumerate(file_list[::util.Z_STEP]):
        img = Image.open(i)
        img = np.asarray(img, dtype=np.float32)
        img = cv2.resize(img, (buf.shape[2], buf.shape[1]), interpolation=cv2.INTER_AREA)
        buf[z, :, :] = img
    return buf

In [ ]:
# もし、既にnumpy配列データのファイルが存在している場合、そちらを読み込む
if npz_data_path.exists():
    saved_data = np.load(npz_data_path)
    data_list = [saved_data['data1'], saved_data['data2']]    
else:
    # チャネルごとに3D画像データを読み込む
    data1 = load_data([str(x) for x in data_path.glob('W0040*C1.tif')])
    data2 = load_data([str(x) for x in data_path.glob('W0040*C2.tif')])
    np.savez_compressed(npz_data_path, data1=data1, data2=data2)
    data_list = [data1, data2]

In [ ]:
# 3D画像データの寸法を計算する
util.init_bounds(data_list[0])

In [ ]:
# volume_listに画像データ(複数チャネル)
obj = k3d.multi_mip(
    volume_list=data_list, 
    color_map_list=util.COLOR_MAP_LIST,
    samples=0, 
    gradient_step=0, 
    alpha_blending=False,
    bounds=(0, util.cx, 0, util.cy, 0, util.cz)
)

In [ ]:
# 3Dビューアの表示エリアを生成
plot = k3d.plot(background_color=0x000000, lighting=0, grid_visible=False, axes_helper=False, height=540)
plot += obj

In [ ]:
# コントロールの準備
util.init_controls(plot, obj, phi=40, distance=(1000, 1, 3000))
fov = 20

### 次のセルを実行したら、3D表示が完了するまでその次のセルを実行しないこと。

In [ ]:
# 3Dビューア及びコントロールの表示
plot.display()
util.display_controls() 

# コントロールの初期値を変更
util.slider_ch[0].value = [172, 900]
util. slider_ch[1].value = [120, 500]

In [ ]:
# FOVやカメラ位置を3D表示に反映する (いったん3D表示してからでないと働かないことがあるので、ここで実行する)
util.refresh()
plot.camera_fov = fov

In [ ]:
util.refresh()

### 動画作成はこちら👇
* 動画作成にはffmpeg・ffmpeg-python・frame_writer2.pyが必要。

In [ ]:
util.display_movie_controls()

In [ ]:
util.input_movie_filename.value = 'tutorial_spheroid_30fps.mp4'

In [ ]:
util.state_store

### スクリーンショットはalpha_blending==Trueのときは正常に得られないので注意。

In [ ]:
plot.fetch_screenshot()

In [ ]:
# ブラウザからサーバのPythonへ転送するのに時間がかかるため、0バイトの場合がある。
len(plot.screenshot)

In [ ]:
import base64, io
png_stream = base64.b64decode(plot.screenshot)
img = Image.open(io.BytesIO(png_stream))
img